In [ ]:
import pandas as pd
import os
import pickle
import datetime
import telegram_send
from multiprocessing import Pool, Lock

In [ ]:
## for multicore machines, define how many cores should be used

In [ ]:
nc=int(7)

In [ ]:
## State if bigrams or trigrams are to be examined

In [ ]:
ng = int(2)

## Load the stemmed business model thesaurus and the bigrams

In [ ]:
## create a python dictionary containing the sets and descriptors for each set
## in the business model thesaurus

In [ ]:
path2bmt='/path/to/stemmed/business/model/thesaurus'

In [ ]:
df=pd.read_excel(path2bmt,index_col=None)

In [ ]:
sets=dict()
for col in df:
    sets[col]=df[col].dropna().tolist()

In [ ]:
## create a list of the filepaths of the files containing the
## text data scraped from the wayback machine

In [ ]:
path2wbm='/path/to/waybackmachine/bigrams'

In [ ]:
dflist=[os.path.join(path2wbm,df) for df in os.listdir(path2wbm) if df.startswith('df')]

## Define the function for tagging the terms in the text that fit the descriptors 

In [ ]:
## here the function is defined that checks if the descriptors 
## appear in the text from the websites and then tags them if appropriate

In [ ]:
def topic_distribution(cell):
    dis=[]
    if pd.isna(cell):
        return None
    for top in sets:  ## loop through the sets in the thesaurus
        c=0
        for val in sets[top]:   # loop through descriptors
            vall=val.strip().split(' ')
            
            if not len(vall) == ng: # ensure only required n-grams are condsidered
                continue
                
            for wrd in cell.split(','):
                wrdl=wrd.strip().split(' ')
                
                if not len(wrdl) == ng:
                    continue
                
                # set 1st specific restraint
                if vall[1] == 'business':
                    if vall[0] == wrdl[0] and vall[1] == wrdl[1]: # tag term if descriptor is contained within term from website text
                        c+=1
                    else:
                        continue
                # 2nd specific restraint
                elif vall[1] == 'handel':
                    if not wrdl[1].endswith('handel'):
                        continue
                    else:
                        if vall[0] in wrdl[0] and vall[1] in wrdl[1]: # tag term if descriptor is contained within term from website text
                            c+=1
                        else:
                            continue
                # 3rd specific restraint
                elif vall[0] == 'power':
                    if not wrdl[0] == vall[0]:
                        continue
                    else:
                        if vall[0] in wrdl[0] and vall[1] in wrdl[1]: # tag term if descriptor is contained within term from website text
                            c+=1
                        else:
                            continue
                # 4th specific restraint
                elif vall[0] == 'self':
                    if not wrdl[0].endswith('self'):
                        continue
                    else:
                        if vall[0] in wrdl[0] and vall[1] in wrdl[1]: # tag term if descriptor is contained within term from website text
                            c+=1
                        else:
                            continue
                # 5th specific restraint
                elif vall[0] == 'gruen':
                    if 'gruend' in wrdl[0]:
                        continue
                    else:
                        if vall[0] in wrdl[0] and vall[1] in wrdl[1]: # tag term if descriptor is contained within term from website text
                            c+=1
                        else:
                            continue
                # 6th specific restraint
                elif vall[0] == 'konventionel' and vall[1] == 'strom' or vall[1] == 'waerm':
                    if 'erzeug' in wrdl[1]:
                        continue
                    else:
                        if vall[0] in wrdl[0] and vall[1] in wrdl[1]: # tag term if descriptor is contained within term from website text
                            c+=1
                        else:
                            continue
                            
                else:
                    if vall[0] in wrdl[0] and vall[1] in wrdl[1]: # tag term if descriptor is contained within term from website text
                        c+=1
                    else:
                        continue
                        
        dis.append(c)

        
    return dis

## Prepare a new dataframe to contain the results of the categorisation

In [ ]:
cols=df.columns.tolist().copy()

In [ ]:
cols.append('year')

In [ ]:
cols.append('website')

In [ ]:
new_df=pd.DataFrame(columns=cols)

In [ ]:
# prepare a list of the files that have been already processed

In [ ]:
already_processed=[]

In [ ]:
with open('already_processed.p','wb') as f:
        pickle.dump(already_processed,f)

## Apply functions to all files

In [ ]:
## here the function is defined that will be applied to each file containing the text from the websites
## This will be applied utilising all available CPUs using the multiprocessing package

In [ ]:
def top_dist(file):
    st = datetime.datetime.now()
    fname=os.path.basename(file)
    print(fname,st)
    
    new_df=pd.DataFrame(columns=cols)

    # load the file with the website text into a dataframe
    df1 = pickle.load(open(file, 'rb'))
    # apply the function defined in section 2 to the dataframe
    dft = df1.applymap(topic_distribution)
    for website in dft.index:
        for col in dft.loc[website].dropna().index:
            
            dfn = pd.DataFrame.from_dict(dict(zip(cols, dft.loc[website, col])), orient='index').T
            dfn['year'] = col
            dfn['website'] = website
            new_df = new_df.append(dfn)
    
    # update the temporary dataframe file with the results
    lock.acquire()
    df=pd.read_csv('test.csv')
    df=df.append(new_df)
    df.to_csv('test.csv',index=False)
    lock.release()
    
    # update the list of files that have been already processed
    lock.acquire()
    with open('already_processed.p','rb') as f:
        ap=pickle.load(f)
        ap.append(fname)
        print(f'{len(ap)} files processed so far')
    
    with open('already_processed.p','wb') as f:
        pickle.dump(ap,f)
    lock.release()
    
    endtime = datetime.datetime.now()
    tt = endtime-st
    print(f'Time taken for {fname} was {tt}')

In [ ]:
## here the functions are defined for assigning the above function
## to individual cores

In [ ]:
def init(l):
    global lock
    lock = l

In [ ]:
def main():
    l = Lock()
    pool = Pool(initializer=init, initargs=(l,),processes=nc)
    pool.map(top_dist, dflist)
    pool.close()
    pool.join()

In [ ]:
## export empty dataframe that will be used to store the results
new_df.to_csv('test.csv',index=False)

In [ ]:
## apply functions to the files
starttime=datetime.datetime.now()
main()
endtime=datetime.datetime.now()

ttt=endtime-starttime
print(f'Total time taken was {ttt}')   

## Export the results

In [ ]:
df=pd.read_csv('test.csv')

In [ ]:
nw=datetime.datetime.now().strftime('%d_%m_%y@%H:%M')

In [ ]:
df.to_excel(f'/home/rory/wbm_downloads/topic_distribution/top_dist_2g_{nw}.xlsx')

In [ ]:
## send notification of successfull completion to telegram account

In [ ]:
telegram_send.send(messages=[f'Finished bigram categorisation\nFilename={nw} \nTotal time taken was {ttt}'])